In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [59]:
from IPython.display import JSON

In [60]:
from googleapiclient.discovery import build

In [71]:
file = open('API_key.txt')
api_key = file.read()
#api_key

In [72]:
channel_ids = ['UC9HGzFGt7BLmWDqooUbWGBg', #Matthew ID
               'UCYqdbnuvx-jJZByotoNW-5Q', # Apollonia
              ]

In [73]:
api_service_name = "youtube"
api_version = "v3"

# Build API client
youtube = build(api_service_name, api_version, developerKey=api_key)

In [74]:
def get_channel_stats(youtube, channel_ids):
    data = []
    
    # create request
    request = youtube.channels().list(part="snippet,contentDetails,statistics", id=channel_ids)
    
    response = request.execute()

    # loop through the channel
    for item in response['items']:
        info = {'ChannelName': item['snippet']['title'],
                'Subscribers': item['statistics']['subscriberCount'],
                'TotalViews': item['statistics']['viewCount'],
                'Videos': item['statistics']['videoCount'],
                'PlaylistID': item['contentDetails']['relatedPlaylists']['uploads'],   
               
               }
        data.append(info)
    return(pd.DataFrame(data))



In [75]:
df = get_channel_stats(youtube, channel_ids)
df

,ChannelName,Subscribers,TotalViews,Videos,PlaylistID
0,Apollonia Ponti,682000,105397023,1044,UUYqdbnuvx-jJZByotoNW-5Q
1,Matthew Hussey,3050000,541531749,942,UU9HGzFGt7BLmWDqooUbWGBg


In [76]:

playlistID = 'UU9HGzFGt7BLmWDqooUbWGBg'
request = youtube.playlistItems().list(part="snippet, contentDetails", playlistId=playlistID, maxResults=50) #maximum value by default
response = request.execute()
JSON(response)


<IPython.core.display.JSON object>

In [77]:
def get_videoIDs(youtube, playlistID):
    video_IDs = []   
    
    request = youtube.playlistItems().list(part="snippet, contentDetails", playlistId=playlistID, maxResults=50) #maximum value by default
    response = request.execute()

     # loop through the items
    for item in response['items']:
        video_IDs.append(item['contentDetails']['videoId'])
                        
    next_page_token = response.get('nextPageToken')
    
    while next_page_token is not None:
        request = youtube.playlistItems().list(part="snippet, contentDetails", playlistId=playlistID, maxResults=50, pageToken=next_page_token) #maximum value by default
        response = request.execute()
    
     # loop through the items
        for item in response['items']:
            video_IDs.append(item['contentDetails']['videoId'])
        
        next_page_token = response.get('nextPageToken')

        
    return video_IDs


In [79]:
# retrieve video list for Matthew
playlistID = df.loc[df['ChannelName'] == 'Matthew Hussey', 'PlaylistID'].iloc[0]
VideoList_mat = get_videoIDs(youtube, playlistID) 

# retrieve video list for Apollonia
playlistID = df.loc[df['ChannelName'] == 'Apollonia Ponti', 'PlaylistID'].iloc[0]
VideoList_apo = get_videoIDs(youtube, playlistID) 
len(VideoList_apo)

1045